In [17]:
import torch
import torchvision
import numpy as np
import pandas as pd

In [20]:
#define hyper-parameter
num_classes = 10
lr = 0.001
batch_size = 100
num_epoch = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#download MNIST
train_data = torchvision.datasets.MNIST(download=True,train=True,
                                        root='E:\\code\\pytorch\\data\\MNIST',
                                        transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST(root='E:\\code\\pytorch\\data\\MNIST',
                                      train=False,download=False,
                                       transform=torchvision.transforms.ToTensor())

#dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,
                                           shuffle=True,num_workers=0)

test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,
                                         shuffle=False,num_workers=0)

#define model
class Convnet(torch.nn.Module):
    def __init__(self,output_classes):
        super(Convnet,self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1,out_channels=16,
                            kernel_size=3,stride=1,padding=1,bias=True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3,stride=2,padding=0))
            
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,
                            stride=1,padding=1,bias=True),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3,stride=2,padding=0))
        self.fc = torch.nn.Linear(6*6*32,num_classes)
        self.softmax = torch.nn.Softmax()
            
    #define forward
    def forward(self,x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.reshape(out.size(0), -1)
        #将多维数据平铺成一维的，为了输入到全连接层中
        out = self.fc(out)
        out = self.softmax(out)
        return out

model = Convnet(num_classes).to(device)
    
#define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

#train model
total_step = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        #forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        #backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%300 == 0:
            print('Epoch:[{}/{}],step: [{}/{}],loss: {:.4f}'.format(epoch+1,num_epoch,i+1,total_step,loss.item()))
            
#test model            
model.eval()            
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        
    print('The test accuracy of the model is {}  %'.format(100 * correct/total))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:[1/1],step: [300/1875],loss: 1.5837
Epoch:[1/1],step: [600/1875],loss: 1.4920
Epoch:[1/1],step: [900/1875],loss: 1.5018
Epoch:[1/1],step: [1200/1875],loss: 1.4908
Epoch:[1/1],step: [1500/1875],loss: 1.5061
Epoch:[1/1],step: [1800/1875],loss: 1.4848
The test accuracy of the model is 98.27  %


pytorch中，池化层也是有kernel_size的，这个是指进行池化的窗口大小，而对于卷积和池化的输出特征图计算上，有很多需要注意的地方，这两个层对于输出特征的计算公式都是output_size = input_size - kernel_size + padding * 2 / stride +1,如果输入的大小是不一样的话，各自计算宽和高。
需要注意的是：在特征图的计算中，如果不能整除，都是要向下取整的。

In [19]:
torch.save(model.state_dict(),'Conv_model.ckpt')